In [1]:
import joblib
from fraudetect.config import load_args_from_json
from fraudetect.dataset import load_data
from fraudetect.preprocessing import get_train_val_split
from pathlib import Path
# from fraudetect.preprocessing import FraudFeatureEngineer, FeatureEncoding
# from fraudetect.dataset import MyDatamodule, load_data
# from fraudetect.config import Arguments
import pandas as pd
from datetime import datetime, date
from sklearn.model_selection import (TimeSeriesSplit,
                                     TunedThresholdClassifierCV)
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import (
    StackingClassifier,VotingClassifier
)
from sklearn.frozen import FrozenEstimator
import os
import json
import numpy as np
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV


In [2]:

clf_path = r"D:\fraud-detection-galsen\runs-optuna\ensemble-trees-3_2025-04-26_23-07_best-run.joblib"
    
run = joblib.load(clf_path)

run # 5 models for each cross-val split

Pipeline(steps=[('feature_engineer',
                 FraudFeatureEngineer(behavioral_drift_cols=['CustomerUID'],
                                      reorder_by=['TX_DATETIME', 'AccountId'],
                                      session_gap_minutes=2448, use_sincos=True,
                                      windows_size_in_days=(1, 3, 7, 30))),
                ('feature_engineer_2',
                 AdvancedFeatureEngineer(cat_encoder_name='count',
                                         cat_encoding_kwargs={'base': 4,
                                                              'drop_invariant': False,
                                                              'handle_missing':...
                                                      'woe_randomized': True,
                                                      'woe_regularization': 1.0,
                                                      'woe_sigma': 0.05},
                                 cat_encoding_method='woe',
                                 imputer_n_neighbors=9, onehot_threshold=6)),
                ('to_df_0', ToDataframe()),
                ('feature_selector', SelectKBest(k=40)),
                ('to_df_2', ToDataframe()),
                ('model',
                 GradientBoostingClassifier(criterion='squared_error',
                                            learning_rate=0.023,
                                            min_samples_leaf=3, n_estimators=17,
                                            subsample=0.85))])

In [5]:
run[0].get_params()

{'add_fft': False,
 'add_fraud_rate_features': True,
 'add_seasonal_features': False,
 'behavioral_drift_cols': ['CustomerUID'],
 'reorder_by': ['TX_DATETIME', 'AccountId'],
 'session_gap_minutes': 2448,
 'spline_degree': 3,
 'spline_n_knots': 6,
 'uid_col_name': 'CustomerUID',
 'uid_cols': ['AccountId', 'CUSTOMER_ID'],
 'use_sincos': True,
 'use_spline': False,
 'windows_size_in_days': (1, 3, 7, 30)}

In [10]:
run[1].get_params()

{'add_amount_value_ratio': True,
 'add_cum_features': True,
 'add_lombscargle_features': False,
 'cat_encoder_name': 'count',
 'cat_encoding_kwargs': {'hash_n_components': 6,
  'handle_missing': 'value',
  'return_df': True,
  'hash_method': 'md5',
  'drop_invariant': False,
  'handle_unknown': 'value',
  'base': 4,
  'woe_randomized': True,
  'woe_sigma': 0.05,
  'woe_regularization': 1.0},
 'col_uid_name': 'CustomerUID',
 'max_period': 72,
 'min_period': 6,
 'n_clusters': 0,
 'n_freqs': 5,
 'reorder_by': ['TX_DATETIME', 'AccountId'],
 'uid_cols': ['AccountId', 'CUSTOMER_ID']}

In [8]:
run[-1].get_params()

{'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'init': None,
 'learning_rate': 0.023,
 'loss': 'log_loss',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 3,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 17,
 'n_iter_no_change': None,
 'random_state': None,
 'subsample': 0.85,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}